In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

base = 'http://mit.spbau.ru'

r = requests.get('http://mit.spbau.ru/students/se')
html = r.text

soup = BeautifulSoup(html, 'html.parser')

names = []
images = []
hrefs = []
for div in soup.find_all('div', {'class': 'field-content alumni-userpic'}):
    for a in div.find_all('a'):
        names.append(a.text)
    for img in div.find_all('img'):
        images.append(img['src'])
        hrefs.append(base + a['href'])

df = pd.DataFrame({'name': [name for name in names if 0 < len(name.split()) < 5], \
                   'image': images, 'url': hrefs})

fields = ['field field-name-field-program field-type-list-text field-label-above',
         'field field-name-field-gradyead field-type-list-integer field-label-inline clearfix',
         'field field-name-field-thesistopic field-type-text field-label-above',
         'field field-name-field-advisor field-type-text field-label-above', 
         'field field-name-field-wherenow field-type-text field-label-above',
         'field field-name-field-before-au field-type-text field-label-inline clearfix']
image_class = 'field field-name-field-alumni-photo field-type-image field-label-hidden'

blocks = []
images = []
for url in df.url:
    try:
        block = []
        r = requests.get(url)
        html = r.text
        soup = BeautifulSoup(html, 'html.parser')
        for field in fields:
            try:
                block.append(soup.find('div', {'class': field}).text)
            except:
                continue
        blocks.append(block)
        images.append(soup.find('div', {'class': image_class}).find('img')['src'])
    except:
        blocks.append([])
        images.append('None')
        print(url)

def get_info(block):
    dicts = [{k: v} for k, v in [line.split(':') for line in block]]
    info = dict(pair for d in dicts for pair in d.items())
    return info

infos = []
for block in blocks:
    infos.append(get_info(block))

df['info'] = infos

df['big_images'] = images



df.head()

http://mit.spbau.ruhttp://bioinf.spbau.ru/sites/default/files/AlexeyGurevich_thesis_fulltext.pdf
http://mit.spbau.ruhttp://bioinf.spbau.ru/sites/default/files/AndreyPrjibelski_thesis_fulltext.pdf


,name,image,url,info,big_images
0,Абрамов Иван Александрович,http://mit.spbau.ru/files/alumni/%D0%90%D0%B1%...,http://mit.spbau.ru/node/906,"{'Программа обучения': ' SE', 'Год окончания':...",http://mit.spbau.ru/files/alumni/%D0%90%D0%B1%...
1,Акимов Евгений Владимирович,http://mit.spbau.ru/files/alumni/%D0%90%D0%BA%...,http://mit.spbau.ru/node/911,"{'Программа обучения': ' SE', 'Год окончания':...",http://mit.spbau.ru/files/alumni/%D0%90%D0%BA%...
2,Бибаев Виталий Игоревич,http://mit.spbau.ru/files/alumni/bibaevsm.jpg,http://mit.spbau.ru/node/914,"{'Программа обучения': ' SE', 'Год окончания':...",http://mit.spbau.ru/files/alumni/bibaev.jpg
3,Васильев Роман Алексеевич,http://mit.spbau.ru/files/alumni/roman.vasilie...,http://mit.spbau.ru/node/907,"{'Программа обучения': ' SE', 'Год окончания':...",http://mit.spbau.ru/files/alumni/roman.vasilie...
4,Гарифуллин Шамиль Раифович,http://mit.spbau.ru/files/alumni/%D0%93%D0%B0%...,http://mit.spbau.ru/node/901,"{'Программа обучения': ' SE', 'Год окончания':...",http://mit.spbau.ru/files/alumni/%D0%93%D0%B0%...


In [1]:
path = 'au_photos'
for idx in range(len(df)):
    if df.big_images[idx] != 'None':
        img_data = requests.get(df.big_images[idx]).content
    else:
        img_data = requests.get(df.image[idx]).content
    with open(os.path.join(path, f'{df.name[idx].split()[0]}.png'), 'wb') as handler:
        handler.write(img_data)

NameError: name 'df' is not defined

# asdsa

In [1]:
# matches = []
# for root, dirnames, filenames in os.walk():
#     for filename in fnmatch.filter(filenames, '*.c'):
#         matches.append(os.path.join(root, filename))
import os
from glob import glob
import re
import face_recognition as FC
from PIL import Image, ImageDraw
path = 'au_photos'
img_lst = glob(os.path.join(path, "*"))

In [87]:
import cv2

class CompareImage(object):

    def __init__(self, image_1_path, image_2_path):
        self.minimum_commutative_image_diff = 1
        self.image_1_path = image_1_path
        self.image_2_path = image_2_path

    def compare_image(self):
        image_1 = cv2.imread(self.image_1_path, 0)
        image_2 = cv2.imread(self.image_2_path, 0)
        commutative_image_diff = self.get_image_difference(image_1, image_2)

        if commutative_image_diff < self.minimum_commutative_image_diff:
            #print ("Matched")
            return commutative_image_diff
        return False #random failure value

    @staticmethod
    def get_image_difference(image_1, image_2):
        first_image_hist = cv2.calcHist([image_1], [0], None, [256], [0, 256])
        second_image_hist = cv2.calcHist([image_2], [0], None, [256], [0, 256])
        
        #метод гистограмм
        img_hist_diff = cv2.compareHist(first_image_hist, second_image_hist, cv2.HISTCMP_BHATTACHARYYA)
        #метод сопоставления с образцом
        img_template_probability_match = cv2.matchTemplate(first_image_hist, second_image_hist, cv2.TM_CCOEFF_NORMED)[0][0]
        img_template_diff = 1 - img_template_probability_match
        
        # в общей разнице между картинками 10% учитывается методом гистограмм,
        # а 90% - метод сопоставления с образцом
        commutative_image_diff = (img_hist_diff / 10) + img_template_diff
        return commutative_image_diff


if __name__ == '__main__':
    for i in range(70):
        compare_image = CompareImage(img_lst[0], img_lst[i])
        image_difference = compare_image.compare_image()
        #print (image_difference)

In [79]:
face_match = {}

for i in img_lst:

    cur_person_name = i[10:-4]
    
    for j in img_lst:
        if i == j:
            print("you don't wanna compare urself with urself")
            continue
            
        unk_person_name = j[10:-4]
        
        compare_image = CompareImage(i, j)
        img_diff = compare_image.compare_image()
        if img_diff == False:
            print(f"{cur_person_name} can never look like {unk_person_name}...")
        else:
            img_sim = abs(1 - img_diff)
            if img_diff < 0.5:
                
                new_value = [unk_person_name, img_sim]
                if face_match.get(cur_person_name):
                    face_match[cur_person_name].append(new_value)
                else:
                    face_match[cur_person_name] = [new_value]
            print(f"{cur_person_name}'s face matches {unk_person_name}'s one with {int(img_sim * 100)}% !")

you don't wanna compare urself with urself
Серебро's face matches Башоров's one with 2%
Серебро doesn't look like Курбанов
Серебро's face matches Бибаев's one with 18%
Серебро's face matches Сулименова's one with 23%
Серебро's face matches Марченко's one with 74%
Серебро doesn't look like Лохару
Серебро's face matches Волкова's one with 13%
Серебро doesn't look like Певзнер
Серебро doesn't look like Кривохатский
Серебро's face matches Пивоваров's one with 4%
Серебро doesn't look like Москвитин
Серебро doesn't look like Пржибельский
Серебро's face matches Атамась's one with 19%
Серебро doesn't look like Ордиян
Серебро's face matches Матченко's one with 50%
Серебро's face matches Тузова's one with 8%
Серебро doesn't look like Шабалин
Серебро doesn't look like Орлов
Серебро doesn't look like Абрамов
Серебро doesn't look like Целовальников
Серебро doesn't look like Соколов
Серебро's face matches Шугаепов's one with 3%
Серебро's face matches Пакудин's one with 16%
Серебро's face matches Бат

Бибаев's face matches Пржибельский's one with 22%
Бибаев's face matches Атамась's one with 0%
Бибаев doesn't look like Ордиян
Бибаев doesn't look like Матченко
Бибаев's face matches Тузова's one with 50%
Бибаев's face matches Шабалин's one with 8%
Бибаев's face matches Орлов's one with 17%
Бибаев doesn't look like Абрамов
Бибаев's face matches Целовальников's one with 24%
Бибаев's face matches Соколов's one with 22%
Бибаев's face matches Шугаепов's one with 23%
Бибаев's face matches Пакудин's one with 19%
Бибаев's face matches Баталов's one with 44%
Бибаев's face matches Савельев's one with 13%
Бибаев's face matches Синай's one with 24%
Бибаев's face matches Колобов's one with 2%
Бибаев's face matches Гуревич's one with 26%
Бибаев's face matches Карташов's one with 15%
Бибаев's face matches Гарифуллин's one with 39%
Бибаев doesn't look like Кудинкин
Бибаев doesn't look like Опейкин
Бибаев's face matches Афонина's one with 24%
Бибаев's face matches Коровин's one with 20%
Бибаев's face m

Лохару doesn't look like Лебедев
Лохару doesn't look like Калакуцкий
Лохару doesn't look like Козлов
Лохару's face matches Крыщенко's one with 7%
Лохару's face matches Прошев's one with 35%
Лохару's face matches Мужиков's one with 22%
Лохару's face matches Устюжанина's one with 13%
Лохару doesn't look like Фролова
Лохару doesn't look like Давыдова
Лохару doesn't look like Ждан
Лохару doesn't look like Александрова
Лохару's face matches Калегин's one with 45%
Лохару doesn't look like Бухонов
Лохару doesn't look like Ломшаков
Лохару doesn't look like Горбунов
Лохару's face matches Хабибуллин's one with 47%
Лохару doesn't look like Исаев
Лохару doesn't look like Васильев
Лохару doesn't look like Козятинский
Лохару doesn't look like Бугаев
Лохару doesn't look like Савенко
Лохару's face matches Николаев's one with 13%
Лохару's face matches Дембская's one with 43%
Лохару doesn't look like Гулецкий
Лохару's face matches Цветков's one with 27%
Лохару's face matches Аманов's one with 9%
Лохару 

Кривохатский doesn't look like Пилюгин
Кривохатский doesn't look like Трофимова
Кривохатский doesn't look like Обедин
Кривохатский doesn't look like Осипов
Кривохатский doesn't look like Лепенькин
Кривохатский doesn't look like Кринкин
Пивоваров's face matches Серебро's one with 4%
Пивоваров's face matches Башоров's one with 9%
Пивоваров doesn't look like Курбанов
Пивоваров's face matches Бибаев's one with 15%
Пивоваров doesn't look like Сулименова
Пивоваров doesn't look like Марченко
Пивоваров doesn't look like Лохару
Пивоваров's face matches Волкова's one with 49%
Пивоваров's face matches Певзнер's one with 47%
Пивоваров doesn't look like Кривохатский
you don't wanna compare urself with urself
Пивоваров doesn't look like Москвитин
Пивоваров doesn't look like Пржибельский
Пивоваров doesn't look like Атамась
Пивоваров doesn't look like Ордиян
Пивоваров doesn't look like Матченко
Пивоваров's face matches Тузова's one with 19%
Пивоваров's face matches Шабалин's one with 62%
Пивоваров's f

Атамась doesn't look like Абрамов
Атамась doesn't look like Целовальников
Атамась doesn't look like Соколов
Атамась's face matches Шугаепов's one with 20%
Атамась doesn't look like Пакудин
Атамась's face matches Баталов's one with 9%
Атамась doesn't look like Савельев
Атамась doesn't look like Синай
Атамась doesn't look like Колобов
Атамась doesn't look like Гуревич
Атамась doesn't look like Карташов
Атамась's face matches Гарифуллин's one with 25%
Атамась doesn't look like Кудинкин
Атамась doesn't look like Опейкин
Атамась doesn't look like Афонина
Атамась doesn't look like Коровин
Атамась doesn't look like Майоров
Атамась doesn't look like Сергеев
Атамась doesn't look like Коваленко
Атамась's face matches Жарков's one with 7%
Атамась's face matches Овчинников's one with 10%
Атамась doesn't look like Новокрещенов
Атамась doesn't look like Веслогузова
Атамась doesn't look like Лазаревич
Атамась's face matches Поляков's one with 15%
Атамась's face matches Лапин's one with 18%
Атамась's 

...

Чаркин's face matches Шабалин's one with 22%
Чаркин's face matches Орлов's one with 24%
Чаркин doesn't look like Абрамов
Чаркин doesn't look like Целовальников
Чаркин's face matches Соколов's one with 6%
Чаркин doesn't look like Шугаепов
Чаркин's face matches Пакудин's one with 13%
Чаркин's face matches Баталов's one with 25%
Чаркин's face matches Савельев's one with 20%
Чаркин's face matches Синай's one with 16%
Чаркин doesn't look like Колобов
Чаркин's face matches Гуревич's one with 13%
Чаркин's face matches Карташов's one with 27%
Чаркин's face matches Гарифуллин's one with 16%
Чаркин doesn't look like Кудинкин
Чаркин doesn't look like Опейкин
Чаркин's face matches Афонина's one with 0%
Чаркин doesn't look like Коровин
Чаркин's face matches Майоров's one with 11%
Чаркин's face matches Сергеев's one with 0%
Чаркин doesn't look like Коваленко
Чаркин doesn't look like Жарков
Чаркин doesn't look like Овчинников
Чаркин doesn't look like Новокрещенов
Чаркин doesn't look like Веслогузова


Сорокин's face matches Серебро's one with 17%
Сорокин doesn't look like Башоров
Сорокин doesn't look like Курбанов
Сорокин's face matches Бибаев's one with 34%
Сорокин's face matches Сулименова's one with 23%
Сорокин doesn't look like Марченко
Сорокин's face matches Лохару's one with 0%
Сорокин's face matches Волкова's one with 7%
Сорокин doesn't look like Певзнер
Сорокин doesn't look like Кривохатский
Сорокин's face matches Пивоваров's one with 16%
Сорокин doesn't look like Москвитин
Сорокин doesn't look like Пржибельский
Сорокин's face matches Атамась's one with 19%
Сорокин's face matches Ордиян's one with 5%
Сорокин doesn't look like Матченко
Сорокин's face matches Тузова's one with 7%
Сорокин's face matches Шабалин's one with 19%
Сорокин's face matches Орлов's one with 12%
Сорокин doesn't look like Абрамов
Сорокин doesn't look like Целовальников
Сорокин doesn't look like Соколов
Сорокин's face matches Шугаепов's one with 33%
Сорокин's face matches Пакудин's one with 5%
Сорокин's fa

Служаев's face matches Пилюгин's one with 10%
Служаев doesn't look like Трофимова
Служаев's face matches Обедин's one with 1%
Служаев's face matches Осипов's one with 49%
Служаев doesn't look like Лепенькин
Служаев doesn't look like Кринкин
Михайлов's face matches Серебро's one with 9%
Михайлов's face matches Башоров's one with 40%
Михайлов doesn't look like Курбанов
Михайлов's face matches Бибаев's one with 31%
Михайлов doesn't look like Сулименова
Михайлов doesn't look like Марченко
Михайлов doesn't look like Лохару
Михайлов's face matches Волкова's one with 53%
Михайлов doesn't look like Певзнер
Михайлов doesn't look like Кривохатский
Михайлов's face matches Пивоваров's one with 11%
Михайлов doesn't look like Москвитин
Михайлов's face matches Пржибельский's one with 57%
Михайлов doesn't look like Атамась
Михайлов doesn't look like Ордиян
Михайлов doesn't look like Матченко
Михайлов's face matches Тузова's one with 71%
Михайлов doesn't look like Шабалин
Михайлов doesn't look like Орл

Обедин doesn't look like Коровин
Обедин's face matches Майоров's one with 0%
Обедин doesn't look like Сергеев
Обедин doesn't look like Коваленко
Обедин doesn't look like Жарков
Обедин doesn't look like Овчинников
Обедин doesn't look like Новокрещенов
Обедин doesn't look like Веслогузова
Обедин doesn't look like Лазаревич
Обедин doesn't look like Поляков
Обедин doesn't look like Лапин
Обедин doesn't look like Афанасьев
Обедин doesn't look like Лебедев
Обедин doesn't look like Калакуцкий
Обедин's face matches Козлов's one with 1%
Обедин doesn't look like Крыщенко
Обедин doesn't look like Прошев
Обедин doesn't look like Мужиков
Обедин doesn't look like Устюжанина
Обедин doesn't look like Фролова
Обедин's face matches Давыдова's one with 3%
Обедин doesn't look like Ждан
Обедин's face matches Александрова's one with 0%
Обедин doesn't look like Калегин
Обедин doesn't look like Бухонов
Обедин's face matches Ломшаков's one with 2%
Обедин doesn't look like Горбунов
Обедин doesn't look like Хаби

Кринкин doesn't look like Шугаепов
Кринкин doesn't look like Пакудин
Кринкин doesn't look like Баталов
Кринкин doesn't look like Савельев
Кринкин's face matches Синай's one with 5%
Кринкин's face matches Колобов's one with 9%
Кринкин doesn't look like Гуревич
Кринкин doesn't look like Карташов
Кринкин doesn't look like Гарифуллин
Кринкин's face matches Кудинкин's one with 43%
Кринкин's face matches Опейкин's one with 85%
Кринкин's face matches Афонина's one with 2%
Кринкин doesn't look like Коровин
Кринкин doesn't look like Майоров
Кринкин's face matches Сергеев's one with 9%
Кринкин doesn't look like Коваленко
Кринкин doesn't look like Жарков
Кринкин doesn't look like Овчинников
Кринкин doesn't look like Новокрещенов
Кринкин doesn't look like Веслогузова
Кринкин's face matches Лазаревич's one with 3%
Кринкин's face matches Поляков's one with 1%
Кринкин doesn't look like Лапин
Кринкин doesn't look like Афанасьев
Кринкин doesn't look like Лебедев
Кринкин doesn't look like Калакуцкий
Кри

### Кому какая фотография подходит

In [86]:
for k,v in face_match.items():
    v.sort(key = lambda x: x[1], reverse = True)
    v = v[0]
    print(k, v)

Серебро ['Марченко', 0.7407031714862682]
Башоров ['Лепенькин', 0.8421042265721532]
Курбанов ['Калегин', 0.5569907998246152]
Бибаев ['Горбунов', 0.6416994773942499]
Сулименова ['Афанасьев', 0.6713687475457895]
Марченко ['Серебро', 0.7407031714862682]
Лохару ['Пакудин', 0.5725364696170756]
Волкова ['Бухонов', 0.8836190566825195]
Певзнер ['Майоров', 0.5345086730428359]
Кривохатский ['Жаворонков', 0.6128419912997741]
Пивоваров ['Козятинский', 0.8607092704746018]
Москвитин ['Цветков', 0.5703895640272487]
Пржибельский ['Афонина', 0.8301243528008139]
Атамась ['Калакуцкий', 0.7655883301687811]
Ордиян ['Бугаев', 0.6433530631652643]
Матченко ['Серебро', 0.5042332880620214]
Тузова ['Михайлов', 0.7175135665041825]
Шабалин ['Осипов', 0.8109460528355663]
Орлов ['Ломшаков', 0.8887339349668929]
Целовальников ['Коровин', 0.5975129398339115]
Соколов ['Пржибельский', 0.7763404310251363]
Шугаепов ['Лапин', 0.7193354809720125]
Пакудин ['Лохару', 0.5725364696170756]
Баталов ['Давыдова', 0.8493590412917131]
